In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import pingouin as pg
import matplotlib.pyplot as plt
import torch as tc
genders = ['female', 'male']
male = ['male']
female = ['female']

In [2]:
their_semantics = [
    'Subject',
    'Measuring station',
    'repetition no.',
    'Bust girth',
    'Waist girth',
    'Hip girth',
    'Thigh girth R',
    'Upper arm girth R',
    'Neck girth',
    'Back neck point to waist',
    'Height (m)',
]
our_semantic = [
    'subject',
    'measures_station',
    'repetition',
    'bust_chest_girth', # 5.3.4
    'waist_girth', # 5.3.10
    'hip_girth', # 5.3.13
    'thigh_girth', # 5.3.20
    'upper_arm_girth', # 5.3.16
    'neck_girth', # 5.3.2
    'neck_waist',
    'stature', # 5.1.1
]

station_list = ["MOVE4D", "Expert", "Our"]

measures = pd.read_pickle(f'data/move4d_cleaned_measures.zip')
measures = measures[their_semantics]
measures.columns = our_semantic
our_measures = pd.read_pickle("data/our_measures.zip")
measures = pd.concat([our_measures, measures]).drop(columns=["gender", "stature"])
measures = measures[measures['measures_station'].isin(["MOVE4D", "Expert measurer 1", "our"])]
measures.replace("Expert measurer 1", "Expert", inplace=True)
measures.replace("our", "Our", inplace=True)

,bust_chest_girth,waist_girth,hip_girth,thigh_girth,upper_arm_girth,neck_girth,neck_waist,repetition,measures_station,subject
0,890.102844,750.670166,1049.499756,628.432495,292.741119,325.192902,418.211090,1,Our,IEEEP2_01
1,895.189270,753.261353,1051.522095,625.345581,294.438354,326.743317,426.129272,2,Our,IEEEP2_01
2,903.245544,682.914734,947.787170,538.580383,274.551392,314.705322,417.136688,1,Our,IEEEP2_03
3,886.251953,664.944397,951.866211,537.981140,274.649323,316.283325,414.106720,2,Our,IEEEP2_03
4,1098.261108,981.130737,1141.172607,673.190918,343.884399,369.490021,431.907532,1,Our,IEEEP2_07
...,...,...,...,...,...,...,...,...,...,...
67,980.984009,757.277832,948.672546,577.736755,321.789185,368.718323,426.727905,2,Our,IEEEP2_67
68,1237.505981,1257.936035,1151.078735,671.114441,363.397705,440.467407,490.968323,1,Our,IEEEP2_70
69,1230.704834,1269.401123,1153.597290,665.347534,362.148895,440.307922,487.573822,2,Our,IEEEP2_70
70,951.453796,878.735229,990.817505,556.553284,318.288269,385.026276,443.301636,1,Our,IEEEP2_72


In [32]:
numeric = ['hip_girth', 'thigh_girth']
data = measures[:144][measures[:144]['repetition'] == 1][numeric].to_numpy() - measures[:144][measures[:144]['repetition'] == 2][numeric].to_numpy()
data

array([[-2.02233887e+00,  3.08691406e+00],
       [-4.07904053e+00,  5.99243164e-01],
       [-2.85559082e+00,  7.12695312e+00],
       [-2.46118164e+00, -1.23181152e+00],
       [ 8.97705078e+00,  4.49865723e+00],
       [-2.14245605e+00, -4.39300537e+00],
       [-8.34484863e+00,  1.57250977e+00],
       [ 9.31396484e-02, -3.48901367e+00],
       [ 6.58203125e+00,  1.82006836e+00],
       [-5.08178711e-01,  8.05969238e-01],
       [ 7.40600586e-01,  3.31475830e+00],
       [ 5.74072266e+00,  3.03063965e+00],
       [ 1.78344727e-01,  1.36395264e+00],
       [-3.64245605e+00,  1.40563965e+00],
       [-7.66357422e-01,  4.21746826e+00],
       [ 7.05560303e+00,  1.89862061e+00],
       [ 7.93481445e+00,  1.04522705e+01],
       [ 5.48175049e+00,  9.48791504e-01],
       [ 1.17395020e+00,  2.31542969e+00],
       [-1.20202637e+00, -2.53338623e+00],
       [-8.90869141e-01,  6.28601074e-01],
       [-1.26263428e+00, -9.68872070e-01],
       [-2.05603027e+00,  2.70751953e+00],
       [-1.

# CALC ICC

In [4]:
all_measures = []
for station in station_list:
    measures_of_station = []
    for measure in our_semantic[3:-1]:
        a = measures[measures['measures_station'] == station][['subject', 'repetition', measure]]
        icc = pg.intraclass_corr(data=a, targets='subject', raters='repetition', ratings=measure)['ICC'].loc[2]
        measures_of_station.append(icc)
    all_measures.append(measures_of_station)
our_icc = pd.DataFrame(all_measures, station_list, our_semantic[3:-1]).sort_index()

In [5]:
a = measures[measures['measures_station'] == station][['subject', 'repetition', measure]]
icc = pg.intraclass_corr(data=a, targets='subject', raters='repetition', ratings=measure, nan_policy='omit')
icc

,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.973184,73.583120,71,72,6.212652e-48,"[0.96, 0.98]"
1,ICC2,Single random raters,0.973191,75.032571,71,71,1.198188e-47,"[0.96, 0.98]"
2,ICC3,Single fixed raters,0.973695,75.032571,71,71,1.198188e-47,"[0.96, 0.98]"
3,ICC1k,Average raters absolute,0.986410,73.583120,71,72,6.212652e-48,"[0.98, 0.99]"
4,ICC2k,Average random raters,0.986413,75.032571,71,71,1.198188e-47,"[0.98, 0.99]"
5,ICC3k,Average fixed raters,0.986672,75.032571,71,71,1.198188e-47,"[0.98, 0.99]"


In [6]:
our_icc.index = ['Expert', "MOVE4D", 'Our']
our_icc

,bust_chest_girth,waist_girth,hip_girth,thigh_girth,upper_arm_girth,neck_girth,neck_waist
Expert,0.984964,0.995512,0.993420,0.985788,0.899936,0.980211,0.908226
MOVE4D,0.994299,0.997657,0.999204,0.998120,0.995792,0.995974,0.973715
Our,0.996868,0.998141,0.998908,0.998231,0.994649,0.995912,0.973695


# CALC SEM

In [7]:
all_measures = []
for station in station_list:
    measures_of_station = []
    for measure in our_semantic[3:-1]:
        a = measures[measures['measures_station'] == station][['subject', 'repetition', measure]]
        icc = pg.intraclass_corr(data=a, targets='subject', raters='repetition', ratings=measure, nan_policy='omit')['ICC'].loc[2]
        sem = a[measure].std() * np.sqrt(1-icc)
        measures_of_station.append(sem)
    all_measures.append(measures_of_station)
our_sem = pd.DataFrame(all_measures, station_list, our_semantic[3:-1]).sort_index()

In [8]:
our_sem.index = ['Expert', "MOVE4D", 'Our']
our_sem.T.round(4)

,Expert,MOVE4D,Our
bust_chest_girth,12.3293,7.9139,5.6561
waist_girth,8.5865,6.4644,5.8106
hip_girth,7.9206,2.7550,3.0873
thigh_girth,6.9722,2.5773,2.6495
upper_arm_girth,12.6139,2.4972,2.5938
neck_girth,4.8045,2.3456,2.3739
neck_waist,9.4215,4.8049,3.7130


# CALC PSD

In [9]:
def calculate_psd(measure):
    data = measures
    f1 = data['repetition'] == 1
    f2 = data['repetition'] == 2
    data = data[ f1 | f2 ][['subject', 'measures_station', 'repetition', measure]]
    data = data.pivot_table(values=measure, index=['subject', 'repetition'], columns="measures_station")
    data = data.dropna()
    

    column_wise = []
    for column1 in data.columns:
        row_wise = []
        for column2 in data.columns:
            if column1 == column2:
                row_wise.append(np.nan)
            else:
                d1 = data[column1]
                d2 = data[column2]
                var = (d1-d2).var()
                sem1 = our_sem[measure].loc[column1]**2
                sem2 = our_sem[measure].loc[column2]**2
                value = np.sqrt(var + (sem1/2) + (sem2/2))
                row_wise.append(value)
        column_wise.append(row_wise)
    data = pd.DataFrame(column_wise, index=data.columns, columns=data.columns)
    data.index.name = None
    return data

In [10]:
neck_result = calculate_psd(measure='neck_girth')
MX = pd.Series({"MOVE/Expert": neck_result.loc["MOVE4D"]["Expert"]})
neck_result = neck_result.loc['Our'][:-1].append(MX)

In [11]:
bust_result = calculate_psd(measure='bust_chest_girth')
MX = pd.Series({"MOVE/Expert": bust_result.loc["MOVE4D"]["Expert"]})
bust_result = bust_result.loc['Our'][:-1].append(MX)

In [12]:
waist_result = calculate_psd(measure='waist_girth')
MX = pd.Series({"MOVE/Expert": waist_result.loc["MOVE4D"]["Expert"]})
waist_result = waist_result.loc['Our'][:-1].append(MX)

In [13]:
hip_result = calculate_psd(measure='hip_girth')
MX = pd.Series({"MOVE/Expert": hip_result.loc["MOVE4D"]["Expert"]})
hip_result = hip_result.loc['Our'][:-1].append(MX)

In [14]:
arm_result = calculate_psd(measure='upper_arm_girth')
MX = pd.Series({"MOVE/Expert": arm_result.loc["MOVE4D"]["Expert"]})
arm_result = arm_result.loc['Our'][:-1].append(MX)

In [15]:
thigh_result = calculate_psd(measure='thigh_girth')
MX = pd.Series({"MOVE/Expert": thigh_result.loc["MOVE4D"]["Expert"]})
thigh_result = thigh_result.loc['Our'][:-1].append(MX)

In [16]:
neck_waist_result = calculate_psd(measure='neck_waist')
MX = pd.Series({"MOVE/Expert": neck_waist_result.loc["MOVE4D"]["Expert"]})
neck_waist_result = neck_waist_result.loc['Our'][:-1].append(MX)

In [17]:
def calculate_mad():
    all_measures = []
    for station in station_list:
        measures_of_station = []
        a = measures[measures['measures_station'] == station]
        f1 = a['repetition'] == 1
        f2 = a['repetition'] == 2
        for measure in our_semantic[3:-1]:
            mean = np.nanmean(abs(a[f1][measure].to_numpy() - a[f2][measure].to_numpy()))
            measures_of_station.append(mean)
        all_measures.append(measures_of_station)
    return pd.DataFrame(all_measures, columns=our_semantic[3:-1], index=['MOVE4D','Expert','Our']).T

In [18]:
styles = [dict(selector="caption",
                       props=[("text-align", "center"),
                              ("font-size", "125%"),
                              ("color", 'black'),
                              ("font-weight", '700')])]

In [19]:
mad_table = calculate_mad()
mad_table = mad_table.style.set_caption("OUR vs MOVE4D MAD statistic result").set_table_styles(styles)
mad_table.format(precision=4)


,MOVE4D,Expert,Our
bust_chest_girth,8.5292,12.5347,6.8866
waist_girth,6.4874,9.1319,5.6992
hip_girth,3.0823,8.8194,3.5551
thigh_girth,2.6278,7.8472,2.9425
upper_arm_girth,2.7248,8.2014,2.6521
neck_girth,2.3442,5.2431,2.6996
neck_waist,5.1012,10.4514,4.2303


In [20]:
sem_table = our_sem.T.round(4)
sem_table = sem_table.style.set_caption("OUR vs MOVE4D SEM statistic result").set_table_styles(styles)
sem_table.format(precision=4)

,Expert,MOVE4D,Our
bust_chest_girth,12.3293,7.9139,5.6561
waist_girth,8.5865,6.4644,5.8106
hip_girth,7.9206,2.7550,3.0873
thigh_girth,6.9722,2.5773,2.6495
upper_arm_girth,12.6139,2.4972,2.5938
neck_girth,4.8045,2.3456,2.3739
neck_waist,9.4215,4.8049,3.7130


In [21]:
data_psd = pd.DataFrame(
    [neck_result, bust_result, waist_result, hip_result, arm_result,thigh_result, neck_waist_result],
    index=our_semantic[3:-1]).round(2)
data_psd.columns = ['Our vs Expert', "Our vs MOVE4D", 'MOVE vs Expert']
data_psd.to_csv("output/our_psd.csv")
data_psd = data_psd.style.set_caption("OUR vs MOVE4D PSD statistic result").set_table_styles(styles)
data_psd.format(precision=2)

,Our vs Expert,Our vs MOVE4D,MOVE vs Expert
bust_chest_girth,12.03,7.44,10.17
waist_girth,22.12,20.73,22.43
hip_girth,23.05,13.26,22.21
thigh_girth,15.03,11.94,14.78
upper_arm_girth,16.62,6.02,15.73
neck_girth,14.58,10.43,11.48
neck_waist,19.98,13.70,20.22
